In [1]:
import pandas as pd
import numpy
data=pd.read_json('data.jsonl',lines=True) #Loading in Arkansas-20200302-text Json File

In [31]:
#Checking if GPU is enabled before another other use
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))


2023-06-05 16:53:52.574734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 16:53:54.638426: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-05 16:53:57.614338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


tf.Tensor(-826.04865, shape=(), dtype=float32)


2023-06-05 16:53:57.959056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-05 16:53:57.959205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-05 16:53:57.971198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-05 16:53:57.971433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-05 16:53:57.971508: I tensorflow/compile

In [3]:
data.columns #columns found

Index(['id', 'url', 'name', 'name_abbreviation', 'decision_date',
       'docket_number', 'first_page', 'last_page', 'citations', 'volume',
       'reporter', 'court', 'jurisdiction', 'frontend_url', 'preview',
       'casebody'],
      dtype='object')

In [4]:
for x in data:
    print(data[x][1]) #checking to see if any data is usable for creating embeddings

11638634
https://api.capapi.org/v1/cases/11638634/
FIKES v. BENTLEY
Fikes v. Bentley
1828-05
Case No. 4,785a
50
50
[{'type': 'official', 'cite': '9 F. Cas. 50'}]
{'url': 'https://api.capapi.org/v1/volumes/32044054565676/', 'barcode': '32044054565676', 'volume_number': '9'}
{'url': 'https://api.capapi.org/v1/reporters/942/', 'full_name': 'Federal Cases', 'id': 942}
{'url': 'https://api.capapi.org/v1/courts/ark-super-ct-1/', 'name_abbreviation': 'Ark. Super. Ct.', 'slug': 'ark-super-ct-1', 'name': 'Superior Court of the Territory of Arkansas', 'id': 9132}
{'name': 'Ark.', 'name_long': 'Arkansas', 'whitelisted': True, 'url': 'https://api.capapi.org/v1/jurisdictions/ark/', 'id': 34, 'slug': 'ark'}
https://cite.capapi.org/f-cas/9/50/11638634/
[]
{'data': {'judges': ['Before ESKRIDGE, JOHNSON, and TRIMBLE, Judges.'], 'attorneys': [], 'opinions': [{'type': 'majority', 'text': 'OPINION OP THE COURT. This is an appeal from the Conway circuit court. The appellant moved for a new trial, on an aff

# **Usable data for embedding creating was found to be under 'casebody', under the data section. The data found is the summary for each trial, the plan is to make embeddings from the summary data and use that for semantic searching.**

In [34]:
# testing out regex script for removing punctuation
import re
sentence=data['casebody'][0]['data']['opinions'][0]['text']
py_opstr = re.sub(r'[^\w\s]','',sentence)
print(py_opstr) 
#script working (need to remove stop words and change to lower case)

OPINION OF THE COIÍRT This is an action of debt brought by the appellee against the appellant in the circuit court of Pulaski county and comes to this court by appeal It appears from the record that the defendant in the court below filed his plea of payment to which the plaintiff replied and the defendant refusing to join issue by adding a similiter a judgment on that account was rendered against him and he now contends that this judgment should be reversed
The judgment although not strictly and formally correct is certainly substantially good and ought not to be reversed at the instance of the appellant who was in fault in not completing the pleadings Admitting the English practice in a case like this to be to strike out the plea and enter judgment by default it is not perceived what advantage it has over the practice heretofore adopt ed by this court in the case of Russell v Flanakin unreported in which a judgment precisely similar was entered The defendant in refusing to join issue 

### Now we need to choose a suitable model, so we choose to select all-MiniLM-L6-v2 as the model , as it gives high embedding and good semantic search performance but also being only 80mb in size.

In [5]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')

/home/devesh/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import nltk                         #getting nltk for text preprocessing
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/devesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
len(data)


59735

#### Text preprocessing (removing stop words,punctuation and lemmatizing). Lemmatization was chosen instead of stemming as most words used in law text is very similar to each other, hence we do not want any data loss which comes along with stemming.

In [9]:
nltk.download('wordnet')
import re
# py_opstr = re.sub(r'[^\w\s]','',py_str)
lmt=WordNetLemmatizer()
s=""
processed=[]
embed=[]
temp=[]
stop=stopwords.words('english')
stop.extend([',','/','.',':',';'])
for i in range(len(data)):
    try:
        if (i%1000==0):
            print("Reached: ",i)
        sentence=data['casebody'][i]['data']['opinions'][0]['text'].lower()
        sentence=lmt.lemmatize(sentence)
        # print(sentence)
        for x in sentence.split(" "):
            if x not in stopwords.words('english'):
                s+=x+" "
        s = re.sub(r'[^\w\s]','',s)
        processed.append([s])               #putting pre processed text into an array
        t=model.encode(s)                   #creating embeddings and pushing them onto an array
        embed.append(t)
        s=""
    except:
        print(i)
        processed.append([0000])            #adding dummy data when we cannot find data in casebody
        temp.append(i)
    # print(len(s))
    # print(len(sentence))
    # print(model.encode(s))
    # print(len(data))

[nltk_data] Downloading package wordnet to /home/devesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Reached:  0
2


In [29]:
# for x in embed:
#     print(x==embed[0])

In [ ]:
len(temp)   #17 values were found to be missing

17

In [ ]:
len(embed)-len(processed)

-17

In [ ]:
safe_array=embed.copy()
safe_array==embed

True

In [ ]:
for x in temp:
    safe_array.insert(x,[])

In [ ]:
safe_array[temp[0]]

[]

In [ ]:
final = pd.DataFrame(
    {'Processed': processed,
     'embeddings': safe_array,
    })
temp_df=pd.DataFrame({'values not taken':temp})             #creating a dataframe of missing values and preprocessed and embedding data

In [ ]:
final.to_csv('final.csv')
temp_df.to_csv('temp.csv')

In [ ]:
chec=pd.read_csv('final.csv')


In [ ]:
chec=chec.drop('Unnamed: 0',axis=1)

In [ ]:
final.to_pickle('final.pkl')
temp_df.to_pickle('temp.pkl')  #these files were never used but were created to check time required for storing data

#### Checking back embeddings values from dataframe lead to some discrepencies, so just to be safe , they were also stored in a numpy object or a npy file.

In [ ]:
import numpy as np
np.save('embeddings.npy',embed)

In [ ]:
q=np.load('embeddings.npy')

In [1]:
import pandas as pd
data=pd.read_csv('final.csv')

In [4]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')

/home/devesh/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Created embeddings again as the previous code had a small problem when increasing value of loop when a miss hit happened looking for data in casebody
### And lights kept going out so couldnt run that code again due to it taking around 2 hrs to complete each time-

In [6]:
embed=[]
for x in range(len(data['Processed'])):
    if (x%1000==0):
        print(x)
    s=model.encode(data['Processed'][x])
    embed.append(s)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000


In [7]:
np.save('embeddings1.npy',embed)

In [13]:
f=np.load('embeddings1.npy')

#### Now, those 17 values which resulted in a miss initially were once again search for with different parameters, they were stored under ['casebody']['data']['head_matter']

In [20]:
import pandas as pd
final=pd.read_csv('final.csv')
data=pd.read_json('data.jsonl',lines=True)
temp=pd.read_csv('temp.csv')

In [21]:
for x in range(17):
    final['Processed'][temp['values not taken'][x]]=data['casebody'][temp['values not taken'][x]]['data']['head_matter'] #checking if all missing values are stored under ['casebody']['data']['head_matter']

/tmp/ipykernel_502/823192048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Processed'][temp['values not taken'][x]]=data['casebody'][temp['values not taken'][x]]['data']['head_matter']


#### Processed data again with the help of temp values that we stored earlier 

In [25]:
import re
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
lmt=WordNetLemmatizer()
s=""
stop=stopwords.words('english')
stop.extend([',','/','.',':',';'])
for i in range(17):
        sentence=final['Processed'][temp['values not taken'][i]].lower()
        sentence=lmt.lemmatize(sentence)
        # print(sentence)
        for x in sentence.split(" "):
            if x not in stopwords.words('english'):
                s+=x+" "
        s = re.sub(r'[^\w\s]','',s)
        final['Processed'][temp['values not taken'][i]]=s

/tmp/ipykernel_502/2430700098.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Processed'][temp['values not taken'][i]]=s


In [27]:
temp
sentence=final['Processed'][temp['values not taken'][0]]
print(sentence)
# sentence=lmt.lemmatize(sentence)

case 4399a
in ellis
hempst 10
superior court territory arkansas
oct 1821
2
reported samuel h hempstead esq  


In [28]:
final.to_csv('final.csv')

In [5]:
temp['values not taken'][0]

950

In [19]:
import numpy as np
l=np.load('embeddings1.npy')

In [23]:
len(temp)

17

Creating embeddings for missing values and storing them in npy file

In [26]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('all-MiniLM-L6-v2')
for x in range(17):
    s=data['casebody'][temp['values not taken'][x]]['data']['head_matter']
    f=model.encode(s)
    l[temp['values not taken'][x]]=f

In [31]:
np.save('embeddings.npy',l)